# Pendulum control test

Install the following packages:

```
python -m pip install pyserial
```

Run stm32-stepper-encoder-control.ino on your STM32 Nucleo board. Connect the USB, change the serial port designator, and run through the cells to test controlling the pendulum.

## Message format

From PC to Arduino (note only commands with * are implemented):

```
| SOF  | CMD  | DAT  | CHK  | EOF  |
| 0xA5 |  xx  | yyyy |  zz  | '\n' |

Commands (xx):
  0x00 - Reserved
  0x01 - ACK
  0x02 - NAK
  0x10 - Get encoder angle
  0x20 - Get stepper angle
  0x21 - Set stepper position as home*
  0x22 - Move stepper home*
  0x23 - Move stepper by yyyy steps*
  0x24 - Move stepper to yyyy steps
  
Data (yyyy): 16-bit signed ingeter

Checksum (zz): XOR of SOF, CMD, and DAT bytes
```

From Arduino to PC:

```
| SOF  | ENC  | STP  | CHK  | EOF  |
| 0xA5 | aaaa | bbbb |  cc  | '\n' |

Encoder position (aaaa): 16-bit unsigned integer

Stepper motor position (bbbb): 16-bit unsigned integer

Checksum (cc): XOR of SOF, CMD, and DAT bytes
```

In [ ]:
import sys
import struct

import serial
import serial.tools.list_ports

In [8]:
# List available serial ports
serial_ports = serial.tools.list_ports.comports()
if serial_ports:
    for port, desc, hwid in serial_ports:
        print("  {} : {} [{}]".format(port, desc, hwid))
else:
    print("No serial ports found")

  COM6 : STMicroelectronics STLink Virtual COM Port (COM6) [USB VID:PID=0483:374B SER=0670FF383034544157091044 LOCATION=1-1:x.2]


In [9]:
# Settings
SERIAL_PORT = "COM6"
BAUD_RATE = 115200

In [6]:
# Message constants
TX_SOF = b'\xA5'
TX_EOF = b'\x0A'   # '\n'
TX_CMD_ACK = b'\x01'
TX_CMD_NAK = b'\x02'
TX_CMD_GET_ENC = b'\x10'
TX_CMD_GET_STP = b'\x20'
TX_CMD_SET_STP_HOME = b'\x21'
TX_CMD_MOV_STP_HOME = b'\x22'
TX_CMD_MOV_STP_BY = b'\x23'
TX_CMD_MOV_STP_TO = b'\x24'

In [7]:
# XOR all the bytes in a message to get the checksum value
def calculate_checksum(msg):
    checksum = 0
    
    # XOR over all bytes
    for b in msg:
        checksum = checksum ^ b
    
    return checksum

In [124]:
# Send message over serial
def send_message(ser, cmd, dat):
    
    # Clip data
    dat = max(min(32767, dat), -32768)
    
    # Build message
    msg = bytearray()
    msg += SOF
    msg += cmd
    msg += dat.to_bytes(2, 'big')
    msg += calculate_checksum(msg).to_bytes(1, 'big')
    msg += EOF
    
    # Send message
    ser.write(msg)

In [10]:
# Open serial port
ser = serial.Serial(SERIAL_PORT, BAUD_RATE)

SerialException: could not open port 'COM6': PermissionError(13, 'Access is denied.', None, 5)

In [172]:
# Write out message
cmd = TX_CMD_MOV_STP_BY
dat = 100
send_message(ser, cmd, dat)

# Receive
rx_buf = b''
if ser.in_waiting > 0:
    
    # Read the receive buffer
    while(ser.in_waiting):
        rx_buf += ser.read()
        
        # Look for newline
        if rx_buf[-1] == 10:
            print(rx_buf)

# TODO:
#  Change "TX" prefix to something like "C2A" and return prefix as "A2C"
#  Finish RX code in Arduino
#  Parse message to move stepper
#  Return enc and stepper angles in A2C message

b'#\x00\x00\x00\x00\r\n'


In [11]:
# Close serial port
ser.close()